In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
print(os.listdir("/kaggle/input/handsignimages/Train"))

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

training_set = train_datagen.flow_from_directory(
    "/kaggle/input/handsignimages/Train",
    target_size = (64,64),
    batch_size = 32,
    class_mode = 'sparse'
)

test_datagen = ImageDataGenerator(
    rescale = 1./255
)
test_set = test_datagen.flow_from_directory(
    "/kaggle/input/handsignimages/Test",
    target_size = (64,64),
    batch_size = 32,
    class_mode = 'sparse'
)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Conv2D

cnn = Sequential()
cnn.add(Conv2D(filters = 32,kernel_size = 3,activation = 'relu',input_shape = [64,64,3]))
cnn.add(MaxPool2D(pool_size = 2,strides = 2))

cnn.add(Conv2D(filters = 32,kernel_size = 3,activation = 'relu'))
cnn.add(MaxPool2D(pool_size = 2,strides = 2))

from tensorflow.keras.layers import Flatten
cnn.add(Flatten())

print("CNN model made ")

In [ ]:
from tensorflow.keras.layers import Dense
cnn.add(Dense(units = 128,activation = 'relu'))
cnn.add(Dense(units=24,activation='softmax'))

In [ ]:
cnn.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
num_classes = len(training_set.class_indices)
print(num_classes)
print(training_set.class_indices)  # Shows class mappings


In [ ]:
cnn.fit(x=training_set,validation_data=test_set,epochs=25)

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = '/kaggle/input/handsignimages/Train/L/10019_L.jpg'
img = image.load_img(img_path, target_size=(64, 64))

img_array = image.img_to_array(img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = cnn.predict(img_array)
predicted_class = np.argmax(prediction, axis=1) 

In [ ]:
class_names = list(training_set.class_indices.keys()) 
print(class_names)
predicted_class_name = class_names[predicted_class[0]]
print(predicted_class_name)

In [ ]:
import matplotlib.pyplot as plt

# Display the image
plt.imshow(img)
plt.title(f"Predicted Class: {predicted_class_name}")
plt.show()
